In [76]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from gensim import models, corpora

load_dotenv()

clean_docs_filepath = os.environ['PATH_TO_CLEAN_DOCS']

In [77]:
# load_data
data = []
for file in os.listdir(clean_docs_filepath):
    with open(os.path.join(clean_docs_filepath, file)) as f:
        data.append(f.read())

In [78]:
# load the trained model, dictionary, and corpus
lda_model = models.LdaModel.load('lda_model.gensim')
dictionary = corpora.Dictionary.load('lda_dictionary.dict')
corpus = corpora.MmCorpus('corpus.mm')

def format_topics_sentences(ldamodel, corpus, texts):
    # Initialize output DataFrame
    sent_topics_df = pd.DataFrame(columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])

    # Get the main topic for each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the dominant topic, percent contribution, and keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # Dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                new_row = pd.DataFrame([[int(topic_num), round(prop_topic, 4), topic_keywords]], columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])
                sent_topics_df = pd.concat([sent_topics_df, new_row], ignore_index=True)
            else:
                break

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Original_Text']

    return sent_topics_df

In [79]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)
# print(len(df_dominant_topic))


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1,0.5588,"lung, child, air, people, symptom, condition, cause, need, help, health",welcome to asthma lung uk breath is an inhalation an exhalation breath is life we re the uk s lung health charity fighting for your right to breathe join us for the great north run 2023 be part of the world s largest half marathon experience the incredible atmosphere and witness the red arrows flying overhead all while joining our fight for lung health register today to secure your place at this iconic event lung conditions a z why we re here we believe that every breath matters whoever sai...
1,1,0,0.9900,"lung, disease, pulmonary, information, condition, cause, benefit, help, work, assembly",asbestos in your home gov uk you can at any time departments agencies and public bodies detailed guidance regulations and rules reports analysis and official statistics asbestos in your home get information about asbestos and its disposal from your local council what you need to know available in england and wales only thank you for your feedback help us improve gov uk don t include personal or financial information like your national insurance number or credit card details this field is for...
2,2,3,0.5982,"asthma, inhaler, symptom, child, help, need, use, medicine, find, treatment",spotting asthma symptoms in your child asthma lung uk spotting asthma symptoms in your child does your child cough wheeze or get breathless know the signs of asthma and when to talk to your child s gp if your child or baby is struggling for breath spotting asthma symptoms in your child what asthma symptoms look like in children and how to spot them if you re worried your child might have asthma but you re not sure whether to talk to your gp about it this video shows the things to look out fo...
3,3,1,0.4859,"lung, child, air, people, symptom, condition, cause, need, help, health",exeter pulmonary fibrosis support group every other month via zoom if anyone is interested in joining the meeting please contact howard almond to register your interest via email find out more about
4,4,0,0.9823,"lung, disease, pulmonary, information, condition, cause, benefit, help, work, assembly",statutory rules and papers laid and presented statutory rules and papers laid and presented statutory rules and papers laid and presented statutory rules and papers laid and presented below are lists of statutory rules and papers that have been laid and presented in the assembly business office read press releases watch live and archived video keep up to date with the assembly contacts for different parts of the assembly
5,5,2,0.7306,"help, people, support, care, find, need, information, child, work, lung",funraisin limitless online event fundraising for nonprofits level up your fundraising advanced fundraising software to fuel your nonprofit s growth and build deeper relationships powering award winning fundraising a complete suite of specialityfundraising features peer to peer fundraising turn participant and donor data into action with our native integrations worldwide nonprofits use funraisin to supercharge their impact i agree to be emailed
6,6,0,0.5367,"lung, disease, pulmonary, information, condition, cause, benefit, help, work, assembly",thinking about the end of your life asthma lung uk thinking about the end of your life it s often hard to think about death but it can make things easier on your loved ones if you make decisions and plan in advance there are a few things that you should know so you can inform those closest to you by making a will you can decide what happens to your property and possessions you can draw up a will yourself but it s best to get legal help because there are certain rules to follow when writing t...
7,7,2,0.9973,"help, people, support, care, find, need, information, child, work, lung",how to stay motivated asthma lung uk how to stay motivated find out how to stay motivated and reach your exercise and movem

In [80]:
# extend column width
pd.set_option('display.max_colwidth', 500)

# inspect documents in Topic 0
df_dominant_topic_copy = df_dominant_topic.copy()

print('Topic 0')
topic_0_docs = df_dominant_topic_copy[df_dominant_topic_copy['Dominant_Topic']==0].sort_values('Topic_Perc_Contrib', ascending=False).reset_index(drop=True)
keywords = topic_0_docs["Keywords"][0]
topic_0_docs = topic_0_docs.drop(['Keywords'], axis=1)
print(f"Keywords for Topic 0: {keywords}")
topic_0_docs['Text'] = topic_0_docs['Text'].str.replace(r'\n', ' ', regex=True) # replace newline chars with whitespace for better readability
print(f"Number of Documents in Topic 0: {len(topic_0_docs)}")
topic_0_docs.head(50)


Topic 0
Keywords for Topic 0: lung, disease, pulmonary, information, condition, cause, benefit, help, work, assembly
Number of Documents in Topic 0: 282


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Text
0,294,0,0.9999,industrial injuries disablement benefits technical guidance gov uk you can at any time departments agencies and public bodies detailed guidance regulations and rules reports analysis and official statistics industrial injuries disablement benefits technical guidance 1 about this guide this is one of several guides that give detailed information about social security benefits it is intended for professional advisers and members of the public who want to know more about benefits for people wh...
1,479,0,0.9995,the northern ireland assembly commission supports the work of the media provides office space and facilitates access within parliament buildings and its immediate environs this media handbook sets out the parameters within which media personnel are facilitated within parliament buildings the handbook also sets out the requirement for all media personnel to adhere to a code of conduct while they are in parliament buildings rules for media personnel accessing parliament buildings can be found ...
2,876,0,0.9992,synopsis an alphabetical list of terms used in the northern ireland assembly click on a letter below to read more about any terms that begin with that letter e f g j k t u v x y z chief whips see also the chief whips are the most senior whips in a political party they ensure that mlas attend plenary sessions when required and vote according to party instructions the clerk assistants office services and supports the work of all committees of the assembly and provides secretariat support to t...
3,783,0,0.9986,how we can help how to keep yourself safe take on a challenge if you have been diagnosed with mesothelioma it is likely that you will be able to claim compensation you will need expert legal advice from a solicitor who is a specialist in mesothelioma claims not all solicitors even those that work in personal injury are specialists in mesothelioma claims a specialist mesothelioma solicitor will help you get the right amount of compensation without charging you for the work that they do so th...
4,718,0,0.9981,list of substances that can cause occupational asthma hse 6 list of substances that can cause occupational asthma list of substances that can cause occupational asthma enzymes that change starch into sugar used in flour milling and bread baking also used in detergents animal feed textile processing and brewing c37 c21 c22 used in the expansion of polymers in the rubber and plastics industries c1 an insect derived dyestuff used for cosmetic and pharmaceutical dyeing and as a food and drink co...
5,874,0,0.9980,select an amount or enter another amount i would like to top up my donation by 4 8 to cover fees for the charity 10 helps us raise awareness of lung disease through leaflets and posters would you like to make this a regular donation just a single donation when should your monthly donation be made a monthly payment will be taken on the 17th day of each month starting today the payments will continue until your payment card expires or until you request a cancellation each confirmation email wi...
6,52,0,0.9970,what causes pulmonary hypertension asthma lung uk what causes pulmonary hypertension there are five main groups of pulmonary hypertension as it can be caused by different things pulmonary hypertension caused by left heart disease pulmonary hypertension caused by lung conditions or lack of oxygen pulmonary hypertension caused by blood clots pulmonary hypertension caused by a range of causes group 1 pulmonary arterial hypertension pah is caused by changes to very small arteries which take bloo...
7,757,0,0.9966,you can at any time departments agencies and public bodies detailed guidance regulations and rules reports analysis and official statistics married couples or those in a civil partnership could save up to 252 a year by claiming marriage allowance families looking for childcare places during february half term could be missing out 

In [81]:
# inspect documents in Topic 1
df_dominant_topic_copy = df_dominant_topic.copy()

print('Topic 1')
topic_1_docs = df_dominant_topic_copy[df_dominant_topic_copy['Dominant_Topic']==1].sort_values('Topic_Perc_Contrib', ascending=False).reset_index(drop=True)
keywords = topic_1_docs["Keywords"][0]
topic_1_docs = topic_1_docs.drop(['Keywords'], axis=1)
print(f"Keywords for Topic 1: {keywords}")
topic_1_docs['Text'] = topic_1_docs['Text'].str.replace(r'\n', ' ', regex=True) # replace newline chars with whitespace for better readability
print(f"Number of Documents in Topic 1: {len(topic_1_docs)}")
topic_1_docs.head(50)

Topic 1
Keywords for Topic 1: lung, child, air, people, symptom, condition, cause, need, help, health
Number of Documents in Topic 1: 286


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Text
0,177,1,0.9996,tips for problems sleeping with cpap or non invasive ventilation hope2sleep charity tips for problems sleeping with cpap or non invasive ventilation for fixed cpap pressures with the ramp feature switched off switch on the machine on your usual pressure and with the air blowing tighten the straps just enough to create a seal turn your head from side to side to ensure no leaks occur for automatic variable pressures check if your machine has a mask fit option which most do these days and switc...
1,121,1,0.9994,skyline charity skydiving and parachuting free charity jumps over the past 24 years many people who make a parachute jump do so whilst raising funds for a worthwhile cause skyline are the largest organisers of sponsored charity jumps in the uk working with well over 3 000 charities who will all pay for your charity skydive in return for you raising a minimum amount of funds for them by how you can take advantage of this use our charity selector below to find your charity or one you would lik...
2,1107,1,0.9992,chest drains great ormond street hospital when we breathe in the air passes from the nose and mouth into the windpipe from there it travels into the airways and into the lungs the airways split into lots and lots of smaller tubes each ending in an alveolus the air blood or fluid inside the pleural space needs to be drained away so that the person finds it easier to breathe a chest drain inserted into the pleural space can drain this air blood or fluid away gradually a chest drain consists of...
3,278,1,0.9992,thames path challenge ultra challenge england s greatest river provides a fantastic backdrop for an unforgettable challenge over 3000 adventurers will join the putney bridge start and the full 100km route heads upstream to henley along the famous towpath it s through richmond hampton court runnymede and windsor with some wonderful scenery to a riverside finish line with 50km 25km distance options also available plus the separate 25km thames bridges trek the moonlight 10 walk through central ...
4,39,1,0.9992,you can at any time departments agencies and public bodies detailed guidance regulations and rules reports analysis and official statistics package leaflet information for the recipient this medicinal product has been given authorisation for temporary supply by the uk department of health and social care and the medicines healthcare products regulatory agency it does not have a marketing authorisation but this temporary authorisation grants permission for the medicine to be used for active ...
5,351,1,0.9991,you can at any time departments agencies and public bodies detailed guidance regulations and rules reports analysis and official statistics package leaflet information for the user this medicine is subject to additional monitoring this will allow quick identification of new safety information you can help by reporting any side effects you may get see the end of for how to report side effects read all of this leaflet carefully before you are vaccinated because it contains important informati...
6,676,1,0.9990,ultra challenge walk jog or run home walk jog or run are you a walker jogger or runner looking for a challenge to test yourself or to get in shape and to achieve something special if so these events are for you whether it s along stunning coastline or trails through the best of britain s countryside your ultra challenge will be unforgettable with full support all the way new friends made optional fundraising for your favourite charity and an amazing sense of fulfilment why not join us on the...
7,423,1,0.9990,skyline where you can parachute jump and skydive over the past 24 years choose your nearest centre with lots of parachute centres across the uk there is sure to be one near to you use the map below to click on the dots near to where you live to find out prices and facilities at each airfield parachute jump prices and locations p

In [82]:
# inspect documents in Topic 2
df_dominant_topic_copy = df_dominant_topic.copy()

print('Topic 2')
topic_2_docs = df_dominant_topic_copy[df_dominant_topic_copy['Dominant_Topic']==2].sort_values('Topic_Perc_Contrib', ascending=False).reset_index(drop=True)
keywords = topic_2_docs["Keywords"][0]
topic_2_docs = topic_2_docs.drop(['Keywords'], axis=1)
print(f"Keywords for Topic 2: {keywords}")
topic_2_docs['Text'] = topic_2_docs['Text'].str.replace(r'\n', ' ', regex=True) # replace newline chars with whitespace for better readability
print(f"Number of Documents in Topic 2: {len(topic_2_docs)}")
topic_2_docs.head(50)

Topic 2
Keywords for Topic 2: help, people, support, care, find, need, information, child, work, lung
Number of Documents in Topic 2: 330


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Text
0,528,2,0.9992,you can at any time departments agencies and public bodies detailed guidance regulations and rules reports analysis and official statistics foreign embassies in the uk details of foreign embassies high commissions and consulates in the uk including heads of mission this file is in an format this file is in an format this file is in an format this file is in an format a list of heads of mission in order of precedence including the date of appointment and details of the role last updated 7 ma...
1,476,2,0.9990,losing faith and feeling lost living with a terminal illness find inspiration and support to talk about dying death and grief where your money goes discover how we ve continued to provide vital services this last year how to get a marie curie nurse what do marie curie nurses do how to find a marie curie helper palliative care knowledge zone find specialist information about palliative care for healthcare professionals help marie curie give people their final wishes with a gift in your will i...
2,878,2,0.9989,questions children may ask living with a terminal illness find inspiration and support to talk about dying death and grief where your money goes discover how we ve continued to provide vital services this last year how to get a marie curie nurse what do marie curie nurses do how to find a marie curie helper palliative care knowledge zone find specialist information about palliative care for healthcare professionals help marie curie give people their final wishes with a gift in your will if y...
3,378,2,0.9988,emotional experiences and concerns living with a terminal illness find inspiration and support to talk about dying death and grief where your money goes discover how we ve continued to provide vital services this last year how to get a marie curie nurse what do marie curie nurses do how to find a marie curie helper palliative care knowledge zone find specialist information about palliative care for healthcare professionals help marie curie give people their final wishes with a gift in your w...
4,418,2,0.9988,connect users to enterprise resources with identity based security controls improve security posture with integrated ddos mitigation threat intelligence and more caching dynamic compression optimized route requests and more secure any user accessing any application on any device in any location network security performance reliability on a global scale write code test and deploy static and dynamic applications on cloudflare s global network enable and simplify multi cloud while reducing dat...
5,678,2,0.9988,it all starts with a name find the perfect domain and make a home for your great idea move your domain to a better home for free and save on your renewals feature packed web hosting your site will love free domain and ssl included powerful hosting with dedicated resources for faster website performance make money from reselling our products we ll support you all the way lightning fast hosting built for the world s most popular content management system get an awesome looking site in no time ...
6,417,2,0.9987,books for and about grieving children living with a terminal illness find inspiration and support to talk about dying death and grief where your money goes discover how we ve continued to provide vital services this last year how to get a marie curie nurse what do marie curie nurses do how to find a marie curie helper palliative care knowledge zone find specialist information about palliative care for healthcare professionals help marie curie give people their final wishes with a gift in you...
7,626,2,0.9986,what to feed young children nhs what to feed young children like the rest of the family your toddler needs to eat a variety of foods here are some tips on the different sorts of food to offer your child plus a few it s best to avoid contain lots of vitamins minerals and fibre it s good to introduce lots of different types from a

In [83]:
# inspect documents in Topic 3
df_dominant_topic_copy = df_dominant_topic.copy()

print('Topic 3')
topic_3_docs = df_dominant_topic_copy[df_dominant_topic_copy['Dominant_Topic']==3].sort_values('Topic_Perc_Contrib', ascending=False).reset_index(drop=True)
keywords = topic_3_docs["Keywords"][0]
topic_3_docs = topic_3_docs.drop(['Keywords'], axis=1)
print(f"Keywords for Topic 3: {keywords}")
topic_3_docs['Text'] = topic_3_docs['Text'].str.replace(r'\n', ' ', regex=True) # replace newline chars with whitespace for better readability
print(f"Number of Documents in Topic 3: {len(topic_3_docs)}")
topic_3_docs.head(50)

Topic 3
Keywords for Topic 3: asthma, inhaler, symptom, child, help, need, use, medicine, find, treatment
Number of Documents in Topic 3: 237


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Text
0,318,3,0.9996,book now from across the uk with round the clock airport services enjoy coach travel to the capital and have a big city adventure for less book return coach tickets now with additional direct services added coach hire national express national express transport solutions based in the midlands enjoy quality coach and minibus hire find our lowest fares earn a minimum of 10 cashback when you shop online the uk s no 1 need inspiration read our blog we re no 1 find out what makes us the uk s fav...
1,1121,3,0.9994,common concerns about asthma medicines asthma lung uk common concerns about asthma medicines find answers to common concerns about taking your asthma medicines what do my inhalers do most people with asthma have two inhalers a preventer and a reliever you take your preventer inhaler every day it keeps the inflammation down in your airways and prevents symptoms you take your reliever inhaler when symptoms come on it quickly opens your airways if you react to triggers or have an asthma attack ...
2,941,3,0.9994,common concerns about your child s medicines asthma lung uk common concerns about your child s medicines here we answer common concerns about your child s asthma medicines my child s inhalers what do my child s inhalers do most children with asthma have two inhalers a preventer and a reliever your child needs their preventer inhaler every day it keeps the inflammation down in their airways and prevents symptoms and asthma attacks your child can take their reliever inhaler when symptoms come ...
3,306,3,0.9994,how to travel by train in europe with a wheelchair or special needs the man in seat 61 train travel in europe for people with special needs to central asia china via explore europe by train with an taking your car by train all about the real on via rail s route of the flying scotsman britain s most scenic route the west highland line buy train tickets passes online at the seat61 buy ferry tickets online at the seat61 book hotels online at the seat61 the man in seat for visiting my site have ...
4,904,3,0.9991,completing an asthma action plan with your patients asthma lung uk completing an asthma action plan with your patients an asthma action plan helps your patients take control of their own asthma find out how to complete your patient s asthma action plan so they can benefit from an effective safe and personalised plan download an adult asthma action plan or children s asthma action plan to complete with your patients we have asthma action plans in 8 languages why does my patient need an asthma...
5,36,3,0.9989,peak flow asthma lung uk find out about peak flow how it is used to diagnose and monitor asthma how to test your peak flow at home and what your scores mean what is peak flow peak flow is a measure of how quickly you can blow air out of your lungs if you have asthma you may sometimes have narrow airways this will make it more difficult to blow the air out of your lungs quickly and can change your peak flow score when is peak flow used peak flow is mainly used to help diagnose and monitor ast...
6,60,3,0.9988,asthma at school and nursery asthma lung uk asthma at school and nursery how to make sure your child is safe at school or nursery prepare for your child going to school or nursery making sure your child stays safe with their asthma at school or nursery can feel a bit daunting your child may be going back to school or nursery for a new term starting school or nursery for the first time or joining a new school or nursery if your child has asthma or suspected asthma you may worry about how to p...
7,927,3,0.9988,how is asthma treated in children asthma lung uk how is asthma treated in children find out how inhalers and other medicines can help your child s asthma symptoms and what support you can get from your child s gp or asthma nurse how asthma medicines help the aim of asthma medicines is to stop asthma symptoms your child s gp or a